### 회귀

#### 문제1 | 항공권 가격 예측

In [1]:
# 데이터 : flight_train.csv, flight_test.csv
# 예측값 : price 가격
# 평가 지표 : RMSE

In [151]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor ### 주목 ### 랜포도 회귀에 사용 가능한 모델 있음
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

train = pd.read_csv('flight_train.csv')
test = pd.read_csv('flight_test.csv')
print(train.shape, test.shape)

# EDA
display(train.head(2))
# display(test.head(2))
# train.info()
# test.info()
# train['price'].describe()
# 결측값 없고 오브젝트 타입이 많음
# display(train.describe(include='O'))
# display(test.describe(include='O'))

# 전처리
target = train.pop('price')
print(train.shape, test.shape)
df = pd.concat([train,test])
print(df.shape)

# 인코딩
for col_name in df.select_dtypes(include='O').columns.to_list():
    le = LabelEncoder()
    df[col_name] = le.fit_transform(df[col_name])
    
# 데이터 분할
train = df.iloc[:len(train)].copy()
test = df.iloc[len(train):].copy()
# display(train.head(2))
# display(test.head(2))
x_train, x_val, y_train, y_val = train_test_split(train, target, test_size=0.2, random_state=0)

# 스케일링
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)
test = scaler.transform(test)

# 모델 학습 및 검증
# model = SGDRegressor(random_state=0)
model = RandomForestRegressor(random_state=0)
model.fit(x_train, y_train)
y_pred = model.predict(x_val)
print(mean_squared_error(y_val, y_pred)**0.5)

# 예측 및 제출
pred = model.predict(test)
result = pd.DataFrame({'pred':pred})
result.to_csv('result.csv', index=False)
# pd.read_csv('result.csv')

(10505, 11) (4502, 10)


,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,Vistara,UK-776,Kolkata,Evening,one,Late_Night,Delhi,Economy,6.58,31,7056
1,Vistara,UK-852,Bangalore,Morning,zero,Morning,Mumbai,Business,1.92,37,20760


(10505, 10) (4502, 10)
(15007, 10)
3782.986566193416


In [152]:
#### 문제2 | 노트북 가격 예측

In [153]:
# 데이터 : laptop_train.csv, laptop_test.csv
# 예측값 : price
# 평가 지표: R2 score

In [154]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import r2_score

train = pd.read_csv('laptop_train.csv')
test = pd.read_csv('laptop_test.csv')
# print(train.shape, test.shape)

# EDA
display(train.head(2))
# display(test.head(2))
# Hard_Disk_Capacity 분리하면 좋을 것 같음
# 오브젝트 타입 인코딩 필요
# print(train.isna().sum())
# print(test.isna().sum())
# 결측값 있음
# train['Price'].describe()

# 전처리
target = train.pop('Price')
# print(train.shape, test.shape)
df = pd.concat([train, test])
# print(df.shape)

# 결측치 처리
# 결측치 많은 컬럼 삭제
df = df.drop(columns=['Model'])
# 'X'로 채우기
df['RAM'] = df['RAM'].astype(str)
df = df.fillna('X')

# Hard_Disk_Capacity 분리하기
df['Hard_Disk_Capacity_1'] = df['Hard_Disk_Capacity'].str.split(' ').str[:2].str.join(' ')
df['Hard_Disk_Capacity_2'] = df['Hard_Disk_Capacity'].str.split(' ').str[-1]
df = df.drop(columns='Hard_Disk_Capacity')

# 인코딩
for col_name in df.select_dtypes(include='O').columns.to_list():
    le = LabelEncoder()
    df[col_name] = le.fit_transform(df[col_name].astype(object))

# 스케일링, 결과보고 다시

# 데이터 분리
train = df.iloc[:len(train)].copy()
test = df.iloc[len(train):].copy()
# print(train.shape, test.shape)
x_train, x_val, y_train, y_val = train_test_split(train, target, test_size=0.2, random_state=0)
# print(x_train.shape, y_train.shape)
# print(x_val.shape, y_val.shape)


# 모델 학습 및 검증
model = RandomForestRegressor(random_state=0)
model.fit(x_train, y_train)
y_pred = model.predict(x_val)
print(r2_score(y_val, y_pred))

# 예측 및 제출
pred = model.predict(test)
result = pd.DataFrame({'pred':pred})
result.to_csv('result.csv', index=False)
# pd.read_csv('result.csv')

,Brand,Model,Series,Processor,Processor_Gen,RAM,Hard_Disk_Capacity,OS,Rating,Price
0,ASUS,VivoBook,15,i3,10th,8.0,512 GB SSD,Windows 11 Home,4.3,37940
1,DELL,Inspiron,NaN,i3,11th,8.0,1 TB HDD,Windows 11 Home,3.7,39040


0.8002583947958681
